In [1]:
import pandas as pd
import numpy as np
import torch
import sklearn
import matplotlib
import torchinfo, torchmetrics

# Check PyTorch access (should print out a tensor)
print(torch.randn(3, 3))

# Check for GPU (should return True)
print(torch.cuda.is_available())

tensor([[ 0.4466, -0.5302,  1.1437],
        [ 0.1549,  0.8512,  1.2717],
        [-0.0126,  0.6572,  0.5627]])
False


In [2]:
print(torch.__version__)

2.7.0


In [3]:
# Crea un tensore casuale
x = torch.randn(3, 3)
print(f"Random tensor:\n{x}")

# Verifica disponibilità acceleratori
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"MPS available: {torch.backends.mps.is_available() if hasattr(torch.backends, 'mps') else 'MPS module not found'}")

# Prova a utilizzare MPS se disponibile
if hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device("mps")
    x = x.to(device)
    print(f"Tensor moved to MPS device: {x.device}")

Random tensor:
tensor([[-0.1471,  0.1878,  1.6349],
        [-1.4021, -0.5336, -0.2488],
        [ 1.6960, -1.8533,  1.0584]])
CUDA available: False
MPS available: True
Tensor moved to MPS device: mps:0


## 🚨Durante il corso sostituire CUDA CON MPS🚨

`device = torch.device("cuda" if torch.cuda.is_available() else "cpu")`

diventerà

`device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")`

### Funzione per rendere universale?

In [4]:
def get_device():
    if torch.backends.mps.is_available():
        return torch.device("mps")
    elif torch.cuda.is_available():
        return torch.device("cuda")
    else:
        return torch.device("cpu")

# Poi usa
device = get_device()

In [5]:
print(device)

mps


## Introduction to Tensors
### Creating tensors

I tensori PyTotch sono creati usando `torch.Tensor()` [documentazione](https://pytorch.org/docs/stable/tensors.html)

In [6]:
# scalar (non ha dimensione)

scalar = torch.tensor(7)
scalar

tensor(7)

In [7]:
scalar.ndim

0

In [8]:
# Get tensor back as Python int
scalar.item()

7

### Vector 
Array **unidimensionale** di numeri. Rappresenta una grandezza con magnitudine e direzione.
**due caratteristiche di un vettore**
* **Magnitudine (o Modulo)**: È la lunghezza o grandezza del vettore. Rappresenta quanto è "lungo" il vettore nello spazio. Per un vettore v in uno spazio n-dimensionale, la magnitudine si calcola come la radice quadrata della somma dei quadrati delle sue componenti:
$||v|| = √(v₁² + v₂² + ... + vₙ²)$
* **Direzione**: È l'orientamento del vettore nello spazio. Indica verso dove punta il vettore ed è determinata dagli angoli che il vettore forma con gli assi di riferimento. Un vettore unitario (con magnitudine 1) nella stessa direzione del vettore originale può essere ottenuto dividendo il vettore per la sua magnitudine:
$û = v / ||v||$

Queste due caratteristiche insieme definiscono completamente un vettore: quanto è lungo (magnitudine) e verso dove punta (direzione). In PyTorch e nel machine learning, queste proprietà sono fondamentali per molte operazioni, come la normalizzazione dei vettori, il calcolo di distanze e similitudini, e l'ottimizzazione dei gradienti.

In [15]:
# vector
vector = torch.tensor([7,7])
vector

tensor([7, 7])

In [16]:
vector.ndim

1

In [17]:
vector.shape

torch.Size([2])

### Matrix
Array **bidimensionale** di numeri organizzati in righe e colonne. Trasforma vettori in altri vettori.
Dimensioni: Una matrice è caratterizzata dal numero di righe e colonne (m×n), che definiscono la sua forma.

In [18]:
MATRIX = torch.tensor([[7, 8], 
                       [9, 10]])
MATRIX

tensor([[ 7,  8],
        [ 9, 10]])

In [19]:
MATRIX.ndim

2

In [20]:
MATRIX[1]

tensor([ 9, 10])

In [22]:
MATRIX.shape

torch.Size([2, 2])

## TENSOR
Generalizzazione **multidimensionale** di vettori e matrici. Un array n-dimensionale di numeri. Un vettore è un tensore di rango 1, una matrice è un tensore di rango 2, mentre un tensore può avere rango 3 o superiore. 

Esempio di tensore 3D: `[[[1, 2], [3, 4]], [[5, 6], [7, 8]]]`.

Nel deep learning, i tensori sono l'unità fondamentale di dati: immagini (tensori 4D), sequenze di testo (tensori 3D), pesi delle reti (tensori di vari ranghi).

In [23]:
TENSOR = torch.tensor([[[1,2,3],
                       [3,6,9],
                       [2,3,4]]])
TENSOR

tensor([[[1, 2, 3],
         [3, 6, 9],
         [2, 3, 4]]])

In [25]:
TENSOR.ndim

3

In [26]:
TENSOR.shape

torch.Size([1, 3, 3])

In [27]:
TENSOR[0]

tensor([[1, 2, 3],
        [3, 6, 9],
        [2, 3, 4]])

In [31]:
TENSOR[0][1]

tensor([3, 6, 9])

In [32]:
TENSOR[0][1][2]

tensor(9)

| **Name** | **What is it?** | **Number of dimensions** | **Lower or upper (usually/example)** |
|----------|------------|--------------------------|-------------------------------------|
| **scalar** | a single number | 0 | Lower (a) | 
| **vector** | a number with direction (e.g. wind speed with direction) but can also have many other numbers | 1 | Lower (y) |
| **matrix** | a 2-dimensional array of numbers | 2 | Upper (Q) |
| **tensor** | an n-dimensional array of numbers | can be any number, a 0-dimension tensor is a scalar, a 1-dimension tensor is a vector | Upper (X) |

## Random Tensors

I tensori randomici sono importanti perchè il modo in cui le reti neurali imparano, è iniziare con un tensore pieno di numeri randomici e poi aggiustano questi numeri per una rappresentazione migliore dei dati.

`Start with random numbers -> look at data -> update random numbers -> look at data -> update random numbers...`

In [34]:
# Create a random tensor of shape(size) (3,4)

random_tensor = torch.rand(3,4)
random_tensor

tensor([[0.1070, 0.0685, 0.2837, 0.4913],
        [0.0442, 0.1371, 0.0126, 0.7032],
        [0.7393, 0.0785, 0.3405, 0.2403]])

In [35]:
random_tensor.ndim

2

In [36]:
random_tensor.shape

torch.Size([3, 4])

In [42]:
# create a random tensor with a similar shape to an image
random_image_size_tensor = torch.rand(size=(3, 224,224)) # colour channels (R,G,B), height, width, 
random_image_size_tensor.shape, random_image_size_tensor.ndim

(torch.Size([3, 224, 224]), 3)

![tensore](image/00-tensor-shape-example-of-image.png)

## Zeros and Ones

In [45]:
# Creating Tensor of all zeros (esempio di mask)
zeros = torch.zeros(size=(3,4))
zeros

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [48]:
# Creating Tensor of all ones
ones = torch.ones(size=(3,4))
ones

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [49]:
ones.dtype

torch.float32

## Creating a range of tensors and tensors-like

In [61]:
# torch.arange(start, end, step)
one_to_ten = torch.arange(start=0,end=11, step=1)
one_to_ten

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [60]:
# Creating a tensor like
ten_zeroes = torch.zeros_like(input=one_to_ten)
ten_zeroes

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Il comando `torch.zeros_like(input=one_to_ten)` crea un nuovo tensore con:

* La stessa forma (dimensioni) del tensore one_to_ten
* Lo stesso tipo di dati (dtype) del tensore one_to_ten
* Lo stesso dispositivo (CPU o GPU) del tensore one_to_ten

Ma con tutti gli elementi inizializzati a zero.
Per esempio:

- Se `one_to_ten` è un vettore di lunghezza 10, ten_zeroes sarà un vettore di lunghezza 10 contenente solo zeri
- Se `one_to_ten` è una matrice 3×4, ten_zeroes sarà una matrice 3×4 contenente solo zeri

**Questo è molto utile quando hai bisogno di creare un tensore con le stesse caratteristiche di un tensore esistente, ma con valori diversi**. Altri metodi simili includono:

- `torch.ones_like()`: come zeros_like ma riempie con 1
- `torch.randn_like()`: stessa forma ma con valori casuali da una distribuzione normale
- `torch.rand_like()`: stessa forma ma con valori casuali uniformi tra 0 e 1
- `torch.empty_like()`: stessa forma ma senza inizializzare i valori (per performance)

Se invece volessi creare un tensore con la stessa forma ma con un dtype o dispositivo diverso, potresti specificarlo:

`pythonten_zeroes = torch.zeros_like(one_to_ten, dtype=torch.float32, device="mps")`

## Tensor Datatypes
- Il `dtype` è per la **precision in computing**
    - **Precision** is the amount of detail used to describe a number.
      The higher the precision value (8, 16, 32), the more detail and hence data
      used to express a number. This matters in deep learning and numerical computing because you're making so many operations, the more detail you have to calculate on, the more compute you have to use. So lower precision datatypes are generally faster to compute on but sacrifice some performance on evaluation metrics like accuracy (faster to compute but less accurate).
- `device` what device is the tensor stored on? (usually GPU or CPU)
- `require_grad` se vogliamo o meno calcolare i gradienti

In [64]:
# Float 32 tensor (è il default)
float_32_tensor = torch.tensor([3.0, 6.0, 9.0],
                              dtype=None, # What datatype is the tensor (e.g. float32 or float16)
                              device=None, # defaults to None, which uses the default tensor type
                              requires_grad=False) # if True, operations performed on the tensor are recorded
float_32_tensor

tensor([3., 6., 9.])

In [63]:
float_32_tensor.dtype

torch.float32

In [65]:
float_16_tensor = float_32_tensor.type(torch.float16)
float_16_tensor

tensor([3., 6., 9.], dtype=torch.float16)

## Getting Tensor Attributes
- check tensor datatype `tensor.dtype`
- check tensor shape `tensor.shape`
- check tensor device(GPU|TPU) `tensor.device`

In [66]:
# Create a tensor
some_tensor = torch.rand(3,4)
some_tensor

tensor([[0.7819, 0.0979, 0.0121, 0.7729],
        [0.7100, 0.4248, 0.3079, 0.2666],
        [0.6290, 0.9386, 0.5207, 0.8704]])

In [67]:
# Find out details about some tensor
print(some_tensor)
print(f"Datatype of tensor: {some_tensor.dtype}")
print(f"Shape of tensor: {some_tensor.shape}")
print(f"Device tensor is on: {some_tensor.device}")

tensor([[0.7819, 0.0979, 0.0121, 0.7729],
        [0.7100, 0.4248, 0.3079, 0.2666],
        [0.6290, 0.9386, 0.5207, 0.8704]])
Datatype of tensor: torch.float32
Shape of tensor: torch.Size([3, 4])
Device tensor is on: cpu


## Manipulating Tensors (Tensor Operations)
Le operazioni sui tensori includono:
- Addizioni
- Sottrazioni
- Moltiplicazioni (element-wise)
- Divisioni
- Matrix Multiplication

In [68]:
# Create a tensor and add 10 to it
tensor = torch.tensor([1,2,3])

tensor + 10

tensor([11, 12, 13])

In [69]:
# multiply tensor by 10
tensor * 10

tensor([10, 20, 30])

In [70]:
# subtract 10
tensor -10

tensor([-9, -8, -7])

In [72]:
# in-built function
torch.mul(tensor,10) # torch.add(tensor,10)

tensor([10, 20, 30])

## Moltiplicazione di Matrici (dot product)
La moltiplicazione di matrici è un'operazione fondamentale in algebra lineare con importanti applicazioni nel machine learning e deep learning. 

Le due regole principali sono:

**La dimensione interna deve combaciare**:

- `(3, 2) @ (3, 2)` won't work
- `(2, 3) @ (3, 2)` will work
- `(3, 2) @ (2, 3)` will work
  
**La matriche risultante avrà la forma delle dimensioni esterne**:

- `(2, 3) @ (3, 2) -> (2, 2)`
- `(3, 2) @ (2, 3) -> (3, 3)`
> Note: "@" in Python is the symbol for matrix multiplication.

![matrix mul](image/matrix_mul.png)
[sito math is fun](https://www.mathsisfun.com/algebra/matrix-multiplying.html)

Ecco in cosa consiste:
**Definizione Formale**
Per moltiplicare due matrici A (di dimensione m×n) e B (di dimensione n×p), il risultato sarà una matrice C (di dimensione m×p) dove:
$C[i,j] = \sum_{k=0}^{n-1} A[i,k] × B[k,j]$
In parole semplici, ogni elemento $C[i,j]$ della matrice risultante è il prodotto scalare della riga i-esima di A con la colonna j-esima di B.

**Caratteristiche Chiave**
- **Non è commutativa**: In generale, A×B ≠ B×A.
- **Dimensioni compatibili**: Per poter moltiplicare due matrici, il numero di colonne della prima matrice deve essere uguale al numero di righe della seconda.
- **Complessità computazionale**: O(n³) per l'algoritmo naïve, ma esistono algoritmi più efficienti come Strassen (O(n^2.8)).
- **Interpretazione geometrica**: Rappresenta una composizione di trasformazioni lineari.

**Esempio Semplice**

Per moltiplicare:

`A = [[1, 2],[3, 4]]`
`B = [[5, 6],[7, 8]]`

Il risultato sarà:

`C = [[19, 22],[43, 50]]`

Dove:
```
C[0,0] = A[0,0]×B[0,0] + A[0,1]×B[1,0] = 1×5 + 2×7 = 19
C[0,1] = A[0,0]×B[0,1] + A[0,1]×B[1,1] = 1×6 + 2×8 = 22
C[1,0] = A[1,0]×B[0,0] + A[1,1]×B[1,0] = 3×5 + 4×7 = 43
C[1,1] = A[1,0]×B[0,1] + A[1,1]×B[1,1] = 3×6 + 4×8 = 50
```

In [73]:
# Element wise multiplication

print(tensor, "*", tensor)
print(f"Equals: {tensor * tensor}")

tensor([1, 2, 3]) * tensor([1, 2, 3])
Equals: tensor([1, 4, 9])


In [74]:
# Matrix Multiplication

torch.matmul(tensor,tensor)

tensor(14)

In [75]:
tensor

tensor([1, 2, 3])

In [76]:
# matrix multiplication by hand
1*1+2*2+3*3

14

In [82]:
## Differenze di calcolo
%time
value = 0
for i in range(len(tensor)):
    value+= tensor[i] * tensor[i]
print(value)

CPU times: user 3 μs, sys: 1e+03 ns, total: 4 μs
Wall time: 6.91 μs
tensor(14)


In [83]:
%time
torch.matmul(tensor, tensor)

CPU times: user 4 μs, sys: 2 μs, total: 6 μs
Wall time: 10 μs


tensor(14)

## 🚨 Tensors shape error

💡 [sito matrix multiplication](http://matrixmultiplication.xyz/) 💡, molto simpatico mostra visivamente come viene effettuata un matrix multiplication

In [156]:
# Shape for matrix multiplication
tensor_A = torch.tensor([[1,2],
                       [3,4],
                       [5,6]])

tensor_B = torch.tensor([[7,10],
                       [8,11],
                       [9,12]])

# torch.mm(tensor_A, tensor_B) # `torch_mm` is = `torch.matmul`
# torch.matmul(tensor_A, tensor_B)

### Transpose
To fix our tensor shape issues, we can manipulate the shape of one of our tensors using a **transpose**

**transpose** inverte gli assi o le dimensioni di un dato vettore. 

In [89]:
tensor_B, tensor_B.shape

(tensor([[ 7, 10],
         [ 8, 11],
         [ 9, 12]]),
 torch.Size([3, 2]))

In [88]:
tensor_B.T, tensor_B.T.shape

(tensor([[ 7,  8,  9],
         [10, 11, 12]]),
 torch.Size([2, 3]))

In [93]:
# the matrix mult funziona quando il tensor_b ha un transpose
print(f"Original shapes: tensor_A = {tensor_A.shape}, tensor_B = {tensor_B.shape}")
print(f"New shapes: tensor_A = {tensor_A.shape}, tensor_B.T = {tensor_B.T.shape}")
print(f"Multiplying: {tensor_A.shape} @ {tensor_B.T.shape} <-- inner dimension must match")
print("Ouput:\n")
output =  torch.matmul(tensor_A, tensor_B.T)

print(output)

print(f"\nOutput shape: {output.shape}")

Original shapes: tensor_A = torch.Size([3, 2]), tensor_B = torch.Size([3, 2])
New shapes: tensor_A = torch.Size([3, 2]), tensor_B.T = torch.Size([2, 3])
Multiplying: torch.Size([3, 2]) @ torch.Size([2, 3]) <-- inner dimension must match
Ouput:

tensor([[ 27,  30,  33],
        [ 61,  68,  75],
        [ 95, 106, 117]])

Output shape: torch.Size([3, 3])


## Finding the min, max, mean, sum, etc (tensor aggregation)

In [9]:
# Create a tensor
x = torch.arange(0,100,10)
x, x.dtype

(tensor([ 0, 10, 20, 30, 40, 50, 60, 70, 80, 90]), torch.int64)

In [99]:
# find min
torch.min(x), x.min()

(tensor(0), tensor(0))

In [100]:
# find max
torch.max(x), x.max()

(tensor(90), tensor(90))

In [107]:
# find the mean
# NOTA: `torch.mean() rechiede un tensore di tipo float32
torch.mean(x.type(torch.float32)), x.type(torch.float32).mean()

(tensor(45.), tensor(45.))

In [108]:
# find the sum

torch.sum(x), x.sum()

(tensor(450), tensor(450))

### find argmax e argmin
ci tornerà utile quando useremo il layer **softmax**

In [120]:
 x

tensor([ 0, 10, 20, 30, 40, 50, 60, 70, 80, 90])

In [116]:
# trova l'indice del valore più grande
torch.argmax(x), x.argmax()

(tensor(9), tensor(9))

In [117]:
x[9]

tensor(90)

In [115]:
# trova l'indice del valore più piccolo
torch.argmin(x), x.argmin()

(tensor(0), tensor(0))

In [118]:
x[0]

tensor(0)

## Reshaping stacking, squeezing and unsqueezing tensors

* **Reshaping** - reshapes an input tensor to a defined shape (**deve essere compatili con la shape iniziale**)
* **View** - Return a view of an input tensor of certain shape but keep the same memory as the original tensor
* **Stacking** - combine multiple tensors on top of each other (**vstack**) or side by side (**hstack**)
* **Squeeze** - remove all `1` dimension from a tensor
* **UnSqueeze** - add  a `1` dimension to a target tensor
* **Permute** - Return a view of the input with dimension permuted (swapped) in a certain way

In [10]:
# Let's create a tensor
import torch
x = torch.arange(1.,10.)
x, x.shape

(tensor([1., 2., 3., 4., 5., 6., 7., 8., 9.]), torch.Size([9]))

### Reshape
Rimodella un tensore di input in una forma definita (deve essere compatibile con la forma iniziale, cioè il numero totale di elementi deve rimanere invariato).

In [11]:
# Add an extra dimension
x_reshaped = x.reshape(1,9)
x_reshaped, x_reshaped.shape

(tensor([[1., 2., 3., 4., 5., 6., 7., 8., 9.]]), torch.Size([1, 9]))

In [12]:
x_reshaped_2 = x.reshape(9,1)
x_reshaped_2, x_reshaped_2.shape

(tensor([[1.],
         [2.],
         [3.],
         [4.],
         [5.],
         [6.],
         [7.],
         [8.],
         [9.]]),
 torch.Size([9, 1]))

### View
Restituisce una vista di un tensore di input con una certa forma ma mantiene la stessa memoria del tensore originale. Le modifiche alla vista modificheranno anche il tensore originale poiché condividono gli stessi dati sottostanti.

**view** condivide la memoria con il tensore originale.

Cambiare `z` cambia `x` in quanto una **view** di un tensore condivide la stessa memoria con l'inpunt originale

In [13]:
# Change the view
z = x.view(1,9)
z, z.shape

(tensor([[1., 2., 3., 4., 5., 6., 7., 8., 9.]]), torch.Size([1, 9]))

In [14]:
# Se cambiamo il primo elemento in z, 
# avverrà lo stesso nel tensore originale
z[:,0] = 5
z,x

(tensor([[5., 2., 3., 4., 5., 6., 7., 8., 9.]]),
 tensor([5., 2., 3., 4., 5., 6., 7., 8., 9.]))

### Stacking
Combina più tensori uno sopra l'altro (vstack) o uno accanto all'altro (hstack). 
* `dim=0` **vstack**
* `dim=1` **hstack**

In [15]:
x_stacked = torch.stack([x,x,x,x], dim = 0) # vstack
x_stacked

tensor([[5., 2., 3., 4., 5., 6., 7., 8., 9.],
        [5., 2., 3., 4., 5., 6., 7., 8., 9.],
        [5., 2., 3., 4., 5., 6., 7., 8., 9.],
        [5., 2., 3., 4., 5., 6., 7., 8., 9.]])

In [16]:
x_stacked = torch.stack([x,x,x,x], dim = 1) # hstack
x_stacked

tensor([[5., 5., 5., 5.],
        [2., 2., 2., 2.],
        [3., 3., 3., 3.],
        [4., 4., 4., 4.],
        [5., 5., 5., 5.],
        [6., 6., 6., 6.],
        [7., 7., 7., 7.],
        [8., 8., 8., 8.],
        [9., 9., 9., 9.]])

### Squeeze & Unsqueeze
`torch.squeeze()` remove all single dimensions from a target tensor

Rimuove tutte le dimensioni di grandezza 1 da un tensore. Ad esempio, trasformare un tensore di forma `[1, 3, 1, 2]` in uno di forma `[3, 2]`.

In [17]:
print(f"Previous tensor: {x_reshaped}")
print(f"Previous Shape: {x_reshaped.shape}")

# Remove extra dimensions
x_squeezed = x_reshaped.squeeze()
print(f"\nNew tensor: {x_squeezed}")
print(f"New Shape: {x_squeezed.shape}")

Previous tensor: tensor([[5., 2., 3., 4., 5., 6., 7., 8., 9.]])
Previous Shape: torch.Size([1, 9])

New tensor: tensor([5., 2., 3., 4., 5., 6., 7., 8., 9.])
New Shape: torch.Size([9])


`torch.unsqueeze()` add a single dimensions to a target tensor at specific dim (dimension)

Aggiunge una dimensione di grandezza 1 a un tensore target. Ad esempio, trasformare un vettore di forma `[3]` in una matrice di forma `[1, 3]` o `[3, 1]`, a seconda dell'indice specificato.

🚀 L'operazione `unsqueeze` è molto utile nel deep learning quando hai bisogno di adattare la forma dei tensori per operazioni specifiche, come **aggiungere una dimensione di batch** o una **dimensione di canale per le immagini**.

In [18]:
print(f"Previous tensor: {x_reshaped}")
print(f"Previous Shape: {x_reshaped.shape}")

# add extra dimensions
x_unsqueezed = x_reshaped.unsqueeze(2)
print(f"\nNew tensor: {x_unsqueezed}")
print(f"New Shape: {x_unsqueezed.shape}")

Previous tensor: tensor([[5., 2., 3., 4., 5., 6., 7., 8., 9.]])
Previous Shape: torch.Size([1, 9])

New tensor: tensor([[[5.],
         [2.],
         [3.],
         [4.],
         [5.],
         [6.],
         [7.],
         [8.],
         [9.]]])
New Shape: torch.Size([1, 9, 1])


#### Come leggere le forme dei tensori
ad esempio `torch.Size([1, 9, 1])` è un tensore 3D con:


* 1 "blocco"
* Contenente 9 "righe"
* Ciascuna "riga" ha 1 solo valore

**Suggerimento visivo:**
Per un tensore di forma `[a, b, c, ...]`:

* Pensa a a come al numero di "scatole"
* Ogni scatola contiene b "righe"
* Ogni riga contiene c "colonne"

### Permute
`torch.permute` - rearrages the dimensions of a target tensor in a specified order

Restituisce una **vista** (condivide la stessa memoria) del tensore di input con le dimensioni permutate (scambiate) in un certo modo. Ad esempio, trasformare un tensore di forma `[64, 3, 224, 224]` (**batch, canali, altezza, larghezza**) in `[64, 224, 224, 3]` per cambiare l'ordine delle dimensioni.

In [19]:
x_original = torch.rand(size=(224,224,3)) # [height, width, colour_channels

# permute the original tensor to rearrrange the axis (or dim) order

x_permuted = x_original.permute(2,0,1) # shifts axis 0->1, 1->2, 2->0


print(f"Previous shape: {x_original.shape}")
print(f"New shape: {x_permuted.shape}" )

Previous shape: torch.Size([224, 224, 3])
New shape: torch.Size([3, 224, 224])


In [20]:
x_original[0,0,0] = 728218
x_original[0,0,0], x_permuted[0,0,0]

(tensor(728218.), tensor(728218.))

## Selecting Data from tensors (indexing)

### Indexing (selecting data from tensors)
Indexing con PyTorch è simile all'indexing con NumPy.

In [25]:
# Create a tensor
import torch
x = torch.arange(1,10).reshape(1,3,3) # 1 blocco con 3 righe, che contengono 3 elementi
x, x.shape

(tensor([[[1, 2, 3],
          [4, 5, 6],
          [7, 8, 9]]]),
 torch.Size([1, 3, 3]))

In [26]:
# Let's index on our new tensor
x[0]

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])

In [27]:
# let's index on middle bracket (dim=1)
x[0][0]

tensor([1, 2, 3])

In [28]:
# let's index on the most inner bracket (last dimension)
x[0][0][0]

tensor(1)

In [29]:
x[0][2][2]

tensor(9)

**Possiamo anche usare `:` per selezionare `all` di una dimensione target**

In [30]:
x[:,0]

tensor([[1, 2, 3]])

In [32]:
# Prendere tutti i valori della dimensione 0 e 1 ma solo index 1 della dimensione 2
x[:,:,1]

tensor([[2, 5, 8]])

In [34]:
# Prendere tutti i valori della dimensione 0 ma solo i valori index 1 della prima e seconda dimensione
x[:,1,1]

tensor([5])

In [35]:
# prendere index 0 della dimensione 0 r 1 e tutti i valori della seconda dimensione
x[0,0,:]

tensor([1, 2, 3])

In [36]:
# index on x to return 9
x[0,2,2]

tensor(9)

In [37]:
# index to return 3,6,9
x[:,:,2]

tensor([[3, 6, 9]])

## PyTorch tensors & Numpy
PyTorch ha delle funzionalità per interagine con NumPy.
* Data in Numpy -> PyTorch Tensor -> `torch.from_numpy(ndarray)`
* Pytorch Tensor -> NumPy -> `torch.Tensor.numpy()`


In [40]:
# Numpy array to tensor
import torch
import numpy as np

array = np.arange(1.0, 8.0)
tensor = torch.from_numpy(array)

array, tensor

(array([1., 2., 3., 4., 5., 6., 7.]),
 tensor([1., 2., 3., 4., 5., 6., 7.], dtype=torch.float64))

🚨 Il tipo di default di numpy è **float64**, quindi quando creiamo un tensore da un'array numpy diventerà float64.
Quando creiamo un tensore il tipo di default è **float32**

In [41]:
array.dtype, tensor.dtype

(dtype('float64'), torch.float64)

In [44]:
tensor_original = torch.arange(1.0,8.0)
print(tensor_original.dtype)

torch.float32


🚨 Se cambiamo l'array da cui abbiamo creato il tensore la modifica non si rifletterà sul tensore creato

In [45]:
array = array + 1
array, tensor

(array([2., 3., 4., 5., 6., 7., 8.]),
 tensor([1., 2., 3., 4., 5., 6., 7.], dtype=torch.float64))

🚨 viceversa quando convertiamo un tensore in un array numpy manterrà il dtype **float32**

In [46]:
# Tensor to Numpy
tensor = torch.ones(7)

numpy_tensor = tensor.numpy()
tensor, numpy_tensor

(tensor([1., 1., 1., 1., 1., 1., 1.]),
 array([1., 1., 1., 1., 1., 1., 1.], dtype=float32))

🚨 Se cambiamo il tensor originale, la modifica non si rifletterà sull'array creato

In [48]:
tensor = tensor + 1 
tensor, numpy_tensor

(tensor([2., 2., 2., 2., 2., 2., 2.]),
 array([1., 1., 1., 1., 1., 1., 1.], dtype=float32))

## PyTorch Reproducibility (trying to take random out of random)

Come imparano le reti neurali in breve:

`start with random numbers -> tensor operations -> update random numbers to try and make them better representations of the data -> again -> again --> agai...`

### **Riproducibilità** (come il **random seed** qui viene chiamato **manual_seed**)
I computer sono deterministici, quindi quando richiamiamo il random in realtà è una pseudo randomness.


In [51]:
import torch

# create 2 random tensor

random_tensor_A = torch.rand(3,4)
random_tensor_B = torch.rand(3,4)

print(random_tensor_A)
print(random_tensor_B)

print(random_tensor_A == random_tensor_B)

tensor([[0.1663, 0.3592, 0.8619, 0.0377],
        [0.8570, 0.0400, 0.2148, 0.1747],
        [0.4930, 0.6332, 0.9646, 0.4574]])
tensor([[0.1665, 0.1006, 0.2354, 0.9164],
        [0.0815, 0.6385, 0.2368, 0.1583],
        [0.8363, 0.0158, 0.7170, 0.1521]])
tensor([[False, False, False, False],
        [False, False, False, False],
        [False, False, False, False]])


In [53]:
# Random but reproducible tensors

import torch

# set random seed
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
random_tensor_C = torch.rand(3,4)

torch.manual_seed(RANDOM_SEED)
random_tensor_D = torch.rand(3,4)

print(random_tensor_C)
print(random_tensor_D)

print(random_tensor_C == random_tensor_D)

tensor([[0.8823, 0.9150, 0.3829, 0.9593],
        [0.3904, 0.6009, 0.2566, 0.7936],
        [0.9408, 0.1332, 0.9346, 0.5936]])
tensor([[0.8823, 0.9150, 0.3829, 0.9593],
        [0.3904, 0.6009, 0.2566, 0.7936],
        [0.9408, 0.1332, 0.9346, 0.5936]])
tensor([[True, True, True, True],
        [True, True, True, True],
        [True, True, True, True]])


## Running tensors and PyTorch objects on the GPUs (computazioni più veloci)
* GPUs = computazione più veloce su numeri, grazie a CUDA + Nvidia hardware + PyTorch
(nel mac MPS)
* Oppure usare Cloud computing (AWS, AZURE etc.)


In [55]:
## Controllare accesso

# Check for GPU (should return True)
print(torch.cuda.is_available())

# Check for GPU (should return True)
print(torch.mps.is_available())

False
True


In [60]:
# Setup device agnostic code 
def get_device():
    if torch.backends.mps.is_available():
        return torch.device("mps")
    elif torch.cuda.is_available():
        return torch.device("cuda")
    else:
        return torch.device("cpu")

# Poi usa
device = get_device()

In [61]:
# count the number of devices
torch.mps.device_count()

1

## Putting tensors and models on the GPU

Vogliamo sia tensori che mdelli sulla GPU per essere più veloci. `to(device)`

🚨 Di default i tensori vengono creati sulla CPU

In [62]:
# create a tensor (default on the CPU)

tensor = torch.tensor([1,2,3])

print(tensor, tensor.device)

tensor([1, 2, 3]) cpu


In [63]:
# spostare i tensori su GPU se disponibile

tensor_on_gpu = tensor.to(device)

tensor_on_gpu

tensor([1, 2, 3], device='mps:0')

### Moving tensor back to the CPU
🚨 Se un tensore è su GPU non possiamo trasformarlo in NumPy array.

Se proviamo `tensor_on_gpu.numpy()` otterremo l'errore

`can't convert mps:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.`

Quindi dobbiamo prima riportarlo sulla CPU con `.cpu()`

In [65]:
tensor_on_gpu.numpy()

TypeError: can't convert mps:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [68]:
tensor_back_on_cpu = tensor_on_gpu.cpu().numpy()
tensor_back_on_cpu

array([1, 2, 3])

## Practise

In [75]:
# Create a random tensor with shape (7, 7).abs
torch.manual_seed(42)
tensor = torch.rand(7,7)
tensor

tensor([[0.8823, 0.9150, 0.3829, 0.9593, 0.3904, 0.6009, 0.2566],
        [0.7936, 0.9408, 0.1332, 0.9346, 0.5936, 0.8694, 0.5677],
        [0.7411, 0.4294, 0.8854, 0.5739, 0.2666, 0.6274, 0.2696],
        [0.4414, 0.2969, 0.8317, 0.1053, 0.2695, 0.3588, 0.1994],
        [0.5472, 0.0062, 0.9516, 0.0753, 0.8860, 0.5832, 0.3376],
        [0.8090, 0.5779, 0.9040, 0.5547, 0.3423, 0.6343, 0.3644],
        [0.7104, 0.9464, 0.7890, 0.2814, 0.7886, 0.5895, 0.7539]])

In [76]:
# Perform a matrix multiplication on the tensor from 2 with another random tensor with shape (1, 7)
torch.manual_seed(42)
tensor_b = torch.rand(1,7)

In [77]:
tensor.shape, tensor_b.shape

(torch.Size([7, 7]), torch.Size([1, 7]))

In [79]:
tensor.matmul(tensor_b.T)

tensor([[3.2618],
        [3.4084],
        [2.4866],
        [1.4525],
        [1.7079],
        [2.7291],
        [2.9204]])

In [84]:
# Create two random tensors of shape (2, 3) and send them both to the GPU (you'll need access to a GPU for this). Set torch.manual_seed(1234)
torch.manual_seed(1234)
tensor_c = torch.rand(2,3)
tensor_d = torch.rand(2,3)

tensor_c, tensor_d

(tensor([[0.0290, 0.4019, 0.2598],
         [0.3666, 0.0583, 0.7006]]),
 tensor([[0.0518, 0.4681, 0.6738],
         [0.3315, 0.7837, 0.5631]]))

In [83]:
tensor_c_on_gpu = tensor_c.to(device)
tensor_d_on_gpu = tensor_d.to(device)
tensor_c_on_gpu, tensor_d_on_gpu

(tensor([[0.0290, 0.4019, 0.2598],
         [0.3666, 0.0583, 0.7006]], device='mps:0'),
 tensor([[0.0518, 0.4681, 0.6738],
         [0.3315, 0.7837, 0.5631]], device='mps:0'))

In [85]:
# Perform a matrix multiplication on the tensors you created
tensor_c_on_gpu.shape, tensor_d_on_gpu.shape

(torch.Size([2, 3]), torch.Size([2, 3]))

In [88]:
dot_product = tensor_c_on_gpu.matmul(tensor_d_on_gpu.T)
print(dot_product)

tensor([[0.3647, 0.4709],
        [0.5184, 0.5617]], device='mps:0')


In [92]:
# Find the maximum and minimum values of the dot_product
print(f"Min value {dot_product.min()}")
print(f"Max value {dot_product.max()}")

Min value 0.3647301495075226
Max value 0.5617256760597229


In [94]:
# Find the maximum and minimum index values of the dot_product
print(f"Min value {dot_product.min()} on index {dot_product.argmin()}")
print(f"Max value {dot_product.max()} on index {dot_product.argmax()}")

Min value 0.3647301495075226 on index 0
Max value 0.5617256760597229 on index 3


In [102]:
# Make a random tensor with shape (1, 1, 1, 10) 
# and then create a new tensor with all the 1 dimensions removed to be left with a tensor of shape (10). 
# Set the seed to 7 when you create it and print out the first tensor 
# and it's shape as well as the second tensor and it's shape.
torch.manual_seed(7)

tensor_f = torch.rand(1,1,1,10)
print(tensor_f.ndim)

tensor_f_no_1dim = tensor_f.squeeze()
print(tensor_f_no_1dim.ndim)

4
1
